In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Installing the specific version of tensorflow 2.3.0

In [2]:
!pip install tensorflow==2.3.0

### Importing the necessary libraries

In [3]:
#Python script for forecasting lightning on 7th April 2014 from 11am to 4pm.
import tensorflow as tf
from tensorflow.keras import backend as K 
import xarray as xr
import numpy as np
from sklearn.preprocessing import StandardScaler
from tensorflow.keras import backend as K

In [4]:
print(tf.__version__)

2.3.0


In [5]:
model_path = "/content/drive/MyDrive/Lightning/saved models"

### Defining the custom metrics that were used during model building.

In [6]:
#Defining the metrics used
def POD(y_true, y_pred):
    ytrue = y_true
    ypred = K.sigmoid(y_pred)
    ypred = K.round(ypred)
    true_positives = K.sum(ytrue * ypred)
    possible_positives = K.sum(ytrue)
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def FAR(y_true, y_pred):
    ytrue = y_true
    ypred = K.sigmoid(y_pred)
    ypred = K.round(ypred)
    true_positives = K.sum(ytrue * ypred)
    predicted_positives = K.sum(ypred)
    precision = true_positives / (predicted_positives + K.epsilon())
    return 1 - precision

def binary_acc(y_true,y_pred):
    ypred = K.sigmoid(y_pred)
    return K.mean(K.equal(y_true, K.round(ypred)), axis=-1)

#Defining the loss function
def weight_loss(y_true,y_pred):  # binary classification
    pw = 16
    ytrue = K.flatten(y_true)
    ypred = K.flatten(y_pred)
    return tf.reduce_mean(tf.nn.weighted_cross_entropy_with_logits(logits=ypred,labels=ytrue,pos_weight=pw))

### Importing the saved model 

In [7]:
model = tf.keras.models.load_model(model_path, custom_objects={'POD':POD, 'FAR':FAR, 'weight_loss':weight_loss, 'binary_acc':binary_acc})

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/layers/core.py:1045: UserWarning: Lightning_Binary_Classifier is not loaded, but a Lambda layer uses it. It may cause errors.
  , UserWarning)


### Framing the test data

In [12]:
data_path = '/content/drive/MyDrive/Lightning/data/pre-processed data/BinaryLightningData.nc'
#Testing data is 2014
ds = xr.open_dataset(data_path).sel(time=slice("2014"))
channels = '/content/drive/MyDrive/Lightning/data/pre-processed data/BinaryLightningData.nc'
step = 6

In [ ]:
dataset={}

for i in channels:
  if i == 'r':
    temp=ds[i].values[:,0,:,:]
  else:
    temp=ds[i].values
  scalers = {}
  for j in range(temp.shape[1]):
    scalers[j] = StandardScaler()
    temp[:, j, :] = scalers[j].fit_transform(temp[:, j, :]) 
  dataset[i]=temp
  print(i)

In [14]:
#Forecasting lightning for 7th april 2014

#Input time stamp will be from 138 to 144.
ip1 = ds['p80.162'].values[138:144]
ip2 = ds['p79.162'].values[138:144]
ip3 = ds['cape'].values[138:144]
ip4 = ds['r'].values[138:144,0,:,:]


ip = np.expand_dims(np.stack([ip1,ip2,ip3,ip4],axis=-1), axis=0)

ip5 = np.expand_dims(np.expand_dims(ds['Amplitude'].values[138:144], axis=0), axis=-1)

ip6 = np.expand_dims(np.expand_dims(np.expand_dims(ds['Amplitude'].values[143], axis=0), axis=-1), axis=1)

In [15]:
#Creating input
input=[ip,ip5,ip6]

print(ip.shape)
print(ip5.shape)
print(ip6.shape)

(1, 6, 29, 33, 4)
(1, 6, 29, 33, 1)
(1, 1, 29, 33, 1)


### Forecasting the lightning

In [ ]:
#Giving the test input to the model.
temp = K.round(K.sigmoid(model(input)))
temp = np.array(temp)
temp = temp.reshape(24,29,33)

### Saving the forecasted data as netcdf file.

In [6]:
#Storing the predicted forecast results as output.

lons = np.arange(73,81.25,0.25) 
lats = np.arange(22,14.75,-0.25)
times = [i for i in range(24)]

predicted_data = xr.Dataset(
    {
      "Amplitude":(
           ("time", "latitude", "longitude"),
           temp,
           dict(
            units = 'Amp',
            long_name= 'Amplitude of lightning',
            ),
       ),
    },
    coords={"longitude": lons, "latitude": lats, "time": times},
    attrs=dict(description="Lightning Prediction"),
)

print('Saving the model output')
predicted_data.to_netcdf('/content/drive/MyDrive/Lightning/data/predicted data/predicted_data.nc')